In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

# 12.3. 사용자 정의 모델과 훈련 알고리즘

### 12.3.1. 사용자 정의 손실함수
- MSE와 MAE에 오버피팅등의 문제, 이상치의 문제 등이 있을 떄, 사용할 수 있는 대표적인 손실함수는 Huber가 있은아, 공식 케라스 API 지원하지 않음. 아래와 같이 구현 가능

In [2]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, square_loss, linear_loss)

- NN에서 tf를 이용하여 모델을 구성할 떄는 되도록 위 예시처럼 모든 과정을 벡터화해야하며, tf 연산만 활용해야함

In [3]:
# model.compile(loss = huber_fn, optimizer = "namdam")
# model.fit(X_tr, y_tr, ...)

- 그런데 이렇게 임의로 지정한 함수는 모델을 저장할 때 문제가됨

### 12.3.2. 사용자 정의 요소를 가진 모델의 저장/로드
- 저장시에는 케라스가 함수 이름을 저장하므로, 사실 별 문제가 안됨
- 다만, 모델을 로딩할 때 함수 이름과 실제 함수를 매핑할 딕셔너리를 전달해야함

In [4]:
# model = keras.models.load_model("model.h5", custom_object = {"huber_fn" : huber_fn})
# # 따로 loss가 이거다, 저거다를 정의하지 않아도 되며, 이름만 일치하면 됨

In [5]:
def create_huber(thr = 1.0): # 에러가 큰 지 작은 지 판단할 수 있는 threshold를 인자로 받는 생성함수를 만들어주기
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < thr
        squared_loss = tf.square(error) / 2
        linear_loss = thr * tf.abs(error) - thr*2*0.5
        return tf.where(is_small_error, square_loss, linear_loss)
    return huber_fn

# model.compile(loss = create_huber(2.0), optimizer = "nadam")

- 위 threshold는 모델 저장시에 저장되지 않으며, 모델 로드시 threshold값을 지정하여 dic 전달 필요

In [6]:
# model = keras.models.load_model("model.h5", custom_objects = {"huber_fn" : create_huber(2.0)})

- 이렇게 threshold를 지정하는 과정이 귀찮다면, 처음 loss function을 정의할 때 keras.losses.Loss 클래스를 상속하고 get_config()를 구현하는 방식이 있음

In [7]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs): # 기본적인 하이퍼파라미터를 **Kwargs로 받은 인자를 부모 클래스 생성자에 전달
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred): # 모든 샘플의 손실을 계산하여 반환
        error = y_true - y_pred
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold*2*0.5
        return tf.where(is_small_error, square_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold" : self.threshold} # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리를 반환

- 위 처럼 클래스를 선언하면, 컴파일 시 이 클래스의 인스턴스를 그대로 이용할 수 있음

In [8]:
# model.compile(loss = HuberLoss(2.), optimizer = "nadam")

- 이렇게 컴파일한 모델을 저장하면 threshold도 함께 저장됨. 처음처럼 클래스 이름과 클래스 자체만 매핑해주면 됨
- 여기서는 선언한 함수의 이름이 아니라 '클래스'명 자체를 매치시켜줘야함

In [9]:
# model = keras.model.load_model("model.h5", custom_objects = {"HuberLoss" : HuberLoss})

### 12.3.3. 활성화 함수, 초기화, 규제, 제한 커스터마이징
- 위와 비슷한 방식으로 커스터마이징 가능

In [10]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0]) + shape[1])
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_reguralizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeors_like(weights), weights)

- 만약 함수가 모델과 함께 저장해야할 하이퍼파라메터를 가지고 있다면, 적절한 클래스를 상속해야함
    - ex : keras.regularizers.Regularizer, keras.constraints.Constraint, keras.initializers.Initializer, keras.layers.Layer 등
- 예를들어 factor 하이퍼파라미터를 저장하는 l1규제를 클래스로 선언해보면 아래와 같음

In [11]:
# Regularizer 공식 문서를 참고해야함
# 얘의 경우 부모 클래스에 생성자와 get_config()메서드가 정의되어있지 않음
# 따라서 **kwargs를 호출하지 않았으며, get_config과정에서 **base_config를 받아오지 않아도 됨

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor" : self.factor}

- call()구현 대상 : 손실, 층, 모델
- \_\_call__() 구현 대상 : 규제, 초기화 제한

### 12.3.4. 사용자 정의 지표
- 지표와 손실의 기본적인 개념은 거의 동일하다고 봐도 됨
    - 다만, 지표의 경우 모든 지점에서 0이어도 되며, 미분이 불가능한 지점이 있어도 됨
    - 또한, 지표는 우리가 모델의 결과를 해석하기 위해 존재하는 것이므로, 사람이 이해할 수 있어야함

- 훈련 중에 케라스는 모델의 지표를 기록하고, 모든 에포크에서 나온 결과를 평균내서 보여줌
    - 근데, 이것이 문제가 있음. 예를 들어 다음과 같은 결과가 있다고 가정

In [12]:
precision = keras.metrics.Precision()

print(precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1]))
print(precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0]))

tf.Tensor(0.8, shape=(), dtype=float32)
tf.Tensor(0.5, shape=(), dtype=float32)


- 실제 모델로 만들어서 에포크별로 위와 같은 결과를 뱉었다면, 정밀도는 40%로 평균을 얻을 수 있음
    - 그러나, 각각은 80%, 0%라지만, 실제로는 전체 8개의 TP+FP중 4개가 TP
        - 즉, 실제 정밀도는 50%인 것
- 모델별로 업데이트 되는 정밀도를 기록하기 위한 것을 스트리밍 지표라고 부름(streaming metric or stateful metric)

In [13]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [14]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [15]:
precision.reset_states() # 계속 점진적으로 업데이트 되는 것을 0으로 리셋

- 스트리밍 지표를 만드려면 keras.metrics.Metric클래스를 상속

In [17]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold = 1.0, **kwargs): # **base_config를 호출해야 하는 애들은 **kwargs를 추가
        super().__init__(**kwargs) #기본 매개변수 처리 ex : dtype
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer = "zeros") # 여러 배치에 걸쳐 지표의 상태를 기록하기 위한 변수를 생성
        self.count = self.add_weight("count", initializer = "zeros")
    def update_state(self, y_true, y_pred, sample_weight = None): # 이 클래스를 함수처럼 이용할 때 호출됨.(생성자 바로 다음에 있으니까)
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric)) #값 자체를 바꿔줘야하므로, tf.Variable의 값을 바꿀 수 있는 assign_add메서드 이용
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32)) # tf.size(y_true)가 정수형이므로 cast로 float32로 수정
    def result(self):
        return self.total/self.count
    def get_config(self): # get_config메서드를 구현하여 threshold 변수를 모델과 함께 저장
        base_config = super().get_config()
        return {**base_config, "threshold" : self.threshold}

### 12.3.5. 사용자 정의 층
- 가끔, 특이한 층을 가진 네트워크를 만들어야할때가 있음
    - 또한, 동일한 층 블럭이 여러 번 반복되는 네트워크를 만들 때는 각각을 하나의 층으로 다루는 것이 편리함

##### 가중치가 필요없는 층
- keras.layesr.Flatten이나 keras.layesr.ReLU와 같은 층은 가중치가 없음
- 이런 애들은 그냥 함수를 만들어서 Labmda층으로 감싸는 방법을 선택할 수 있음

In [18]:
# 입력에 지수함수를 적용하는 층의 적용
exponential_layer = keras.layers.Lambda(lambda x : tf.exp(x)) # 마치 pandas에 apply lambda를 쓰듯

- 위 처럼 만든 층은 시퀀셜 API나 함수형 API, 서브클래싱 API에서 일반적인 층처럼 이용할 수도 있으며, 활성화 함수로 이용할 수도 있음

##### 가중치가 필요한 층(상태가 있는 층)
- keras.layers.Layer를 상속해야함

In [19]:
# Dense의 간소화된 버전
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation = None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape): # 가중치마다 add_weight()메서드를 호출하여 층으 ㅣ변수를 만드는 것으로, 층이 처음 사용될 때 호출됨
        self.kernel = self.add_weight(
            name = "kernel", shape = [batch_input_shape[-1], self.units],
            initializer = "glorot_normal")
        self.bias = self.add_weight(
            name = "bias", shape = [self.units], initializer = "zeros")
        super().build(batch_input_shape) # 그리고, 마지막에 반드시 부모의 build()메서드를 호출해야하므로 super로 부모의 build를 불러옴
                      
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias) # 이 층에 필요한 연산을 수행함
            
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units]) # 이 층의 출력 크기를 반환
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units" : self.units,
               "activation" : keras.activations.serialize(self.activation)}

##### 여러 입력을 받는 층
- concatenate와 같은 층을 만드려면 call() 메서드에 모든 입력이 포함된 튜플을 매개변수로 전달해야하며, compute_output_shape() 메서드의 매개변수도 각 입력의 배치 크기르 담은 튜플이어야함

##### 여러 출력을 반환하는 층
- call()메서드가 출력의 리스트를 반환해야하며, compute_output_shape()메서드는 출력마다 하나씩 배치 출력 크기의 리스트를 반환해야함

In [20]:
# 두 개의 입력과 세 개의 출력을 만드는 층

class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X # 튜플 형태로 들어온 X값을 X1, X2에 분배
        return [X1 + X2, X1 * X2, X1 / X2] #세 개의 값을 반환
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1] # 세 개의 값을 브로드캐스팅

##### 훈련과 테스트에서 다르게 동작하는 층
- Dropout이나 BatchNormalization과 같이 훈련과 테스트에서 다르게 동작하는 층이 필요하다면 call()메서드에 training인자를 추가하여 훈련인지 테스트인지 결정

In [21]:
# 훈련 중에 가우시안 잡음을 추가하고 테스트 시에는 아무것도 하지 않는 층

class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, X, trianing = None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev = self.stddev)
            return X + noise
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

### 12.3.6. 사용자 정의 모델
- 자세한 설명은 일단 생략하고, 코드만 확인

In [22]:
class ResidualBlock(keras.layers.Layer): # 층 구현
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation = "elu", kernel_initializer = "he_normal") for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [23]:
class ResidualRegressor(keras.Model): # 서브 클래스 API로 모델 정의
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation = "elu", kernel_initializer = "he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
        
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

- Model 클래스는 Layer 클래스의 서브 클래스이므로 모델을 층처럼 정의할 수 있음
    - 하지만 Model 클래스는 compile, fit, evaluate, predict, get_layers, save 메서드가 있음